# ODIBI Framework - Phase 2: Orchestration

**What's new:** Dependency graphs, Pipeline executor, Engine system, End-to-end pipelines!

This notebook demonstrates the orchestration layer that makes pipelines actually run.

In [1]:
# Setup
import sys
from pathlib import Path

project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"✅ Project root: {project_root}")

✅ Project root: d:\odibi


In [3]:
# Imports
import pandas as pd
import tempfile
import shutil
from odibi.config import PipelineConfig, NodeConfig, ReadConfig, WriteConfig, TransformConfig
from odibi.graph import DependencyGraph
from odibi.pipeline import Pipeline
from odibi.connections import LocalConnection
from odibi.registry import transform, FunctionRegistry

print("✅ All imports successful!")

✅ All imports successful!


---

## 1. Dependency Graph

The graph builder analyzes node dependencies and determines execution order.

In [ ]:
# Test 1.1: Simple linear dependency chain
nodes = [
    NodeConfig(
        name="load",
        read=ReadConfig(connection="local", format="csv", path="input.csv")
    ),
    NodeConfig(
        name="transform",
        depends_on=["load"],
        transform=TransformConfig(steps=["SELECT * FROM load"])
    ),
    NodeConfig(
        name="save",
        depends_on=["transform"],
        write=WriteConfig(connection="local", format="csv", path="output.csv")
    )
]

graph = DependencyGraph(nodes)
execution_order = graph.topological_sort()

print("Execution order:")
for i, node_name in enumerate(execution_order, 1):
    print(f"  {i}. {node_name}")

Execution order:
  1. load1
  2. transform2
  3. save3


In [7]:
# Test 1.2: Parallel nodes (diamond pattern)
nodes = [
    NodeConfig(name="root", read=ReadConfig(connection="local", format="csv", path="a.csv")),
    NodeConfig(name="branch_a", depends_on=["root"], transform=TransformConfig(steps=["SELECT * FROM root"])),
    NodeConfig(name="branch_b", depends_on=["root"], transform=TransformConfig(steps=["SELECT * FROM root"])),
    NodeConfig(name="merge", depends_on=["branch_a", "branch_b"], transform=TransformConfig(steps=["SELECT * FROM branch_a"]))
]

graph = DependencyGraph(nodes)

# Get execution layers (for parallel execution)
layers = graph.get_execution_layers()

print("Execution layers (nodes in same layer can run in parallel):")
for i, layer in enumerate(layers, 1):
    print(f"  Layer {i}: {', '.join(layer)}")

Execution layers (nodes in same layer can run in parallel):
  Layer 1: root
  Layer 2: branch_a, branch_b
  Layer 3: merge


In [9]:
# Test 1.3: Visualize the graph
print(graph.visualize())

Dependency Graph:

Layer 1:
  - root

Layer 2:
  - branch_a (depends on: root)
  - branch_b (depends on: root)

Layer 3:
  - merge (depends on: branch_a, branch_b)



In [10]:
# Test 1.4: Cycle detection (this should fail)
try:
    circular_nodes = [
        NodeConfig(name="node1", depends_on=["node2"], read=ReadConfig(connection="local", format="csv", path="a.csv")),
        NodeConfig(name="node2", depends_on=["node1"], transform=TransformConfig(steps=["SELECT * FROM node1"]))
    ]
    graph = DependencyGraph(circular_nodes)
    print("❌ Should have detected cycle!")
except Exception as e:
    print("✅ Cycle detected:")
    print(f"   {str(e)}")

✅ Cycle detected:
   ✗ Dependency error: Circular dependency detected
  Cycle detected: node1 → node2 → node1


In [11]:
# Test 1.5: Get dependencies and dependents
nodes = [
    NodeConfig(name="a", read=ReadConfig(connection="local", format="csv", path="a.csv")),
    NodeConfig(name="b", depends_on=["a"], transform=TransformConfig(steps=["SELECT * FROM a"])),
    NodeConfig(name="c", depends_on=["b"], transform=TransformConfig(steps=["SELECT * FROM b"]))
]

graph = DependencyGraph(nodes)

print(f"Dependencies of 'c': {graph.get_dependencies('c')}")
print(f"Dependents of 'a': {graph.get_dependents('a')}")
print(f"Independent nodes: {graph.get_independent_nodes()}")

Dependencies of 'c': {'a', 'b'}
Dependents of 'a': {'c', 'b'}
Independent nodes: ['a']


---

## 2. Pipeline Execution

The pipeline executor runs nodes in the correct order, handles failures, and tracks results.

In [12]:
# Setup: Create test data
test_dir = tempfile.mkdtemp()
test_path = Path(test_dir)

# Create sample CSV
sample_data = pd.DataFrame({
    "id": [1, 2, 3, 4, 5],
    "product": ["A", "B", "A", "C", "B"],
    "amount": [100, 200, 150, 300, 250]
})
sample_data.to_csv(test_path / "sales.csv", index=False)

print(f"✅ Test data created at: {test_path}")
print("\nSample data:")
print(sample_data)

✅ Test data created at: C:\Users\henry\AppData\Local\Temp\tmp8x8d5oxv

Sample data:
   id product  amount
0   1       A     100
1   2       B     200
2   3       A     150
3   4       C     300
4   5       B     250


In [13]:
# Test 2.1: Simple read-only pipeline
pipeline_config = PipelineConfig(
    pipeline="simple_read",
    nodes=[
        NodeConfig(
            name="load_sales",
            read=ReadConfig(connection="local", format="csv", path="sales.csv")
        )
    ]
)

connections = {"local": LocalConnection(base_path=str(test_path))}
pipeline = Pipeline(pipeline_config, connections=connections)

results = pipeline.run()

print("Pipeline Results:")
print(f"  Completed: {results.completed}")
print(f"  Failed: {results.failed}")
print(f"  Skipped: {results.skipped}")
print(f"  Duration: {results.duration:.4f}s")

# Check the loaded data
loaded_data = pipeline.context.get("load_sales")
print(f"\n✅ Loaded {len(loaded_data)} rows")
print(loaded_data.head())

Pipeline Results:
  Completed: ['load_sales']
  Failed: []
  Skipped: []
  Duration: 0.0106s

✅ Loaded 5 rows
   id product  amount
0   1       A     100
1   2       B     200
2   3       A     150
3   4       C     300
4   5       B     250


In [ ]:
# Test 2.2: Pipeline with transform function
# Clear registry first
FunctionRegistry._functions.clear()
FunctionRegistry._signatures.clear()

@transform
def filter_high_value(context, source: str, min_amount: float = 150):
    """Filter for high-value transactions."""
    df = context.get(source)
    return df[df["amount"] >= min_amount]

@transform
def calculate_total(context, source: str):
    """Calculate total amount."""
    df = context.get(source)
    df = df.copy()
    df["total"] = df["amount"]  # Simple total for demo
    return df

pipeline_config = PipelineConfig(
    pipeline="transform_pipeline",
    nodes=[
        NodeConfig(
            name="load",
            read=ReadConfig(connection="local", format="csv", path="sales.csv")
        ),
        NodeConfig(
            name="filter",
            depends_on=["load"],
            transform=TransformConfig(
                steps=[
                    {"function": "filter_high_value", "params": {"source": "load", "min_amount": 200}}
                ]
            )
        ),
        NodeConfig(
            name="calculate",
            depends_on=["filter"],
            transform=TransformConfig(
                steps=[
                    {"function": "calculate_total", "params": {"source": "filter"}}
                ]
            )
        )
    ]
)

pipeline = Pipeline(pipeline_config, connections=connections)
results = pipeline.run()

print("Pipeline Results:")
print(f"  Completed: {results.completed}")
print(f"  Duration: {results.duration:.4f}s")

# Check final result
final_data = pipeline.context.get("calculate")
print(f"\n✅ Final result has {len(final_data)} rows (filtered from {len(sample_data)})")
print(final_data)

In [14]:
# Test 2.3: Pipeline with write node
pipeline_config = PipelineConfig(
    pipeline="full_etl",
    nodes=[
        NodeConfig(
            name="load",
            read=ReadConfig(connection="local", format="csv", path="sales.csv")
        ),
        NodeConfig(
            name="save",
            depends_on=["load"],
            write=WriteConfig(
                connection="local",
                format="parquet",
                path="output.parquet",
                mode="overwrite"
            )
        )
    ]
)

pipeline = Pipeline(pipeline_config, connections=connections)
results = pipeline.run()

print("Pipeline Results:")
print(f"  Completed: {results.completed}")

# Verify file was created
output_file = test_path / "output.parquet"
if output_file.exists():
    print(f"\n✅ Output file created: {output_file}")
    saved_data = pd.read_parquet(output_file)
    print(f"   Saved {len(saved_data)} rows")
else:
    print("❌ Output file not found")

Pipeline Results:
  Completed: ['load', 'save']

✅ Output file created: C:\Users\henry\AppData\Local\Temp\tmp8x8d5oxv\output.parquet
   Saved 5 rows


In [15]:
# Test 2.4: Failure handling
@transform
def failing_function(context, source: str):
    """This function intentionally fails."""
    raise ValueError("Intentional failure for testing")

pipeline_config = PipelineConfig(
    pipeline="failure_test",
    nodes=[
        NodeConfig(name="success1", read=ReadConfig(connection="local", format="csv", path="sales.csv")),
        NodeConfig(
            name="fail",
            depends_on=["success1"],
            transform=TransformConfig(
                steps=[{"function": "failing_function", "params": {"source": "success1"}}]
            )
        ),
        NodeConfig(
            name="dependent",
            depends_on=["fail"],
            transform=TransformConfig(steps=["SELECT * FROM fail"])
        ),
        NodeConfig(name="success2", read=ReadConfig(connection="local", format="csv", path="sales.csv"))
    ]
)

pipeline = Pipeline(pipeline_config, connections=connections)
results = pipeline.run()

print("Pipeline Results (with failure):")
print(f"  ✅ Completed: {results.completed}")
print(f"  ❌ Failed: {results.failed}")
print(f"  ⏭ Skipped: {results.skipped}")

print("\nObservations:")
print("  - success1 completed (runs first)")
print("  - fail node failed (intentional)")
print("  - dependent skipped (because fail failed)")
print("  - success2 completed (independent of failure)")

Pipeline Results (with failure):
  ✅ Completed: ['success1', 'success2']
  ❌ Failed: ['fail']
  ⏭ Skipped: ['dependent']

Observations:
  - success1 completed (runs first)
  - fail node failed (intentional)
  - dependent skipped (because fail failed)
  - success2 completed (independent of failure)


In [ ]:
# Test 2.5: Run single node with mock data
@transform
def double_amounts(context, source: str):
    """Double all amounts."""
    df = context.get(source)
    df = df.copy()
    df["amount"] = df["amount"] * 2
    return df

pipeline_config = PipelineConfig(
    pipeline="test",
    nodes=[
        NodeConfig(
            name="process",
            transform=TransformConfig(
                steps=[{"function": "double_amounts", "params": {"source": "input"}}]
            )
        )
    ]
)

pipeline = Pipeline(pipeline_config, connections={})

# Run with mock data
mock_df = pd.DataFrame({"id": [1, 2], "amount": [100, 200]})
result = pipeline.run_node("process", mock_data={"input": mock_df})

print("Single Node Execution:")
print(f"  Success: {result.success}")
print(f"  Duration: {result.duration:.4f}s")

output = pipeline.context.get("process")
print("\nInput:")
print(mock_df)
print("\nOutput (amounts doubled):")
print(output)

In [16]:
# Test 2.6: Validate pipeline without running
pipeline_config = PipelineConfig(
    pipeline="validation_test",
    nodes=[
        NodeConfig(name="a", read=ReadConfig(connection="local", format="csv", path="a.csv")),
        NodeConfig(name="b", depends_on=["a"], transform=TransformConfig(steps=["SELECT * FROM a"])),
        NodeConfig(name="c", depends_on=["b"], transform=TransformConfig(steps=["SELECT * FROM b"]))
    ]
)

pipeline = Pipeline(pipeline_config, connections=connections)
validation = pipeline.validate()

print("Validation Results:")
print(f"  Valid: {validation['valid']}")
print(f"  Node count: {validation['node_count']}")
print(f"  Execution order: {validation['execution_order']}")
if validation['errors']:
    print(f"  Errors: {validation['errors']}")
if validation['warnings']:
    print(f"  Warnings: {validation['warnings']}")

Validation Results:
  Valid: True
  Node count: 3
  Execution order: ['a', 'b', 'c']


In [17]:
# Cleanup
shutil.rmtree(test_dir)
print("✅ Test directory cleaned up")

✅ Test directory cleaned up


---

## Summary

**Phase 2 accomplishments:**

✅ **Dependency Graph** - Analyzes dependencies, detects cycles, plans execution  
✅ **Pipeline Executor** - Runs nodes in order, handles failures, tracks results  
✅ **Engine System** - Abstracts read/write/transform operations  
✅ **Connection System** - Abstracts data sources/destinations  
✅ **End-to-End Pipelines** - Everything works together!

**What you can do now:**
- Build real data pipelines with YAML configs
- Create reusable transform functions with `@transform`
- Read/write CSV, Parquet, JSON
- Execute SQL queries (with DuckDB)
- Handle failures gracefully
- Debug with single-node execution

**Next:** CLI tools, more formats, production features!

In [20]:
import inspect
import pandas as pd

In [26]:
func = pd.DataFrame
sig = inspect.signature(func)
print(func.__name__)
print(sig)

DataFrame
(data=None, index: 'Axes | None' = None, columns: 'Axes | None' = None, dtype: 'Dtype | None' = None, copy: 'bool | None' = None) -> 'None'
